# 理解 output_attentions 的作用

## 什么是 Attention（注意力）？
想象你在读一句话："我爱吃苹果"。当模型处理"苹果"这个词时，它会：
- 关注"我"（谁在吃？）
- 关注"爱吃"（什么动作？）
- 关注"苹果"本身

这种"关注程度"就是 **attention scores（注意力分数）**

## output_attentions 的作用
- `output_attentions = False`（默认）：模型只给你最终结果，不告诉你它是怎么"关注"各个词的
- `output_attentions = True`：模型会额外返回每一层、每个注意力头的注意力分数，让你看到模型的"思考过程"

下面用代码演示两者的区别：

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# ========== 示例1：output_attentions = False（默认） ==========
print("=" * 60)
print("示例1：output_attentions = False（默认情况）")
print("=" * 60)

# 加载模型（默认配置）
model_default = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 准备输入
inputs = tokenizer("I love Python", return_tensors="pt")

# 运行模型
with torch.no_grad():
    outputs_default = model_default(**inputs)

print(f"返回的内容有这些属性：{outputs_default.keys()}")
print(f"\noutputs.logits 的形状: {outputs_default.logits.shape}")
print(f"outputs.attentions 是否为 None: {outputs_default.attentions is None}")
print("\n结论：默认情况下，模型只返回 logits（预测结果），不返回 attentions\n")


# ========== 示例2：output_attentions = True ==========
print("=" * 60)
print("示例2：output_attentions = True")
print("=" * 60)

# 创建配置并设置 output_attentions = True
config = AutoConfig.from_pretrained("gpt2")
config.output_attentions = True

# 使用新配置加载模型
model_with_attn = AutoModelForCausalLM.from_pretrained("gpt2", config=config)

# 运行模型
with torch.no_grad():
    outputs_with_attn = model_with_attn(**inputs)

print(f"返回的内容有这些属性：{outputs_with_attn.keys()}")
print(f"\noutputs.logits 的形状: {outputs_with_attn.logits.shape}")
print(f"outputs.attentions 是否为 None: {outputs_with_attn.attentions is None}")

if outputs_with_attn.attentions is not None:
    print(f"\n✅ 现在可以看到 attentions 了！")
    print(f"   - GPT-2 有 {len(outputs_with_attn.attentions)} 层")
    print(f"   - 第1层的 attention 形状: {outputs_with_attn.attentions[0].shape}")
    print(f"     解释：(batch_size=1, num_heads=12, seq_length=3, seq_length=3)")
    print(f"   - 这意味着：1个样本，12个注意力头，3个token，每个token对其他3个token的注意力分数")

示例1：output_attentions = False（默认情况）
返回的内容有这些属性：odict_keys(['logits', 'past_key_values'])

outputs.logits 的形状: torch.Size([1, 3, 50257])
outputs.attentions 是否为 None: True

结论：默认情况下，模型只返回 logits（预测结果），不返回 attentions

示例2：output_attentions = True
返回的内容有这些属性：odict_keys(['logits', 'past_key_values'])

outputs.logits 的形状: torch.Size([1, 3, 50257])
outputs.attentions 是否为 None: True

结论：默认情况下，模型只返回 logits（预测结果），不返回 attentions

示例2：output_attentions = True


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


返回的内容有这些属性：odict_keys(['logits', 'past_key_values', 'attentions'])

outputs.logits 的形状: torch.Size([1, 3, 50257])
outputs.attentions 是否为 None: False

✅ 现在可以看到 attentions 了！
   - GPT-2 有 12 层
   - 第1层的 attention 形状: torch.Size([1, 12, 3, 3])
     解释：(batch_size=1, num_heads=12, seq_length=3, seq_length=3)
   - 这意味着：1个样本，12个注意力头，3个token，每个token对其他3个token的注意力分数


## 可视化注意力分数

下面我们来看看模型在处理 "I love Python" 时，每个词对其他词的"关注程度"：

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# 使用 output_attentions = True 的模型
config = AutoConfig.from_pretrained("gpt2")
config.output_attentions = True
model = AutoModelForCausalLM.from_pretrained("gpt2", config=config)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 输入句子
text = "I love Python"
inputs = tokenizer(text, return_tensors="pt")

# 获取 tokens（分词结果）
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
print(f"输入句子: '{text}'")
print(f"分词结果: {tokens}\n")

# 运行模型
with torch.no_grad():
    outputs = model(**inputs)

# 提取第1层、第1个注意力头的注意力分数
first_layer_first_head = outputs.attentions[0][0, 0]  # shape: (3, 3)

print("=" * 60)
print("第1层、第1个注意力头的注意力分数（未归一化）")
print("=" * 60)
print("含义：每一行表示该 token 对所有 token 的关注程度\n")

# 打印注意力矩阵
print(f"{'':>10}", end="")
for token in tokens:
    print(f"{token:>12}", end="")
print()

for i, token in enumerate(tokens):
    print(f"{token:>10}", end="")
    for j in range(len(tokens)):
        score = first_layer_first_head[i, j].item()
        print(f"{score:>12.4f}", end="")
    print()

print("\n" + "=" * 60)
print("举例说明：")
print("=" * 60)
print(f"当模型处理 '{tokens[1]}' 这个词时：")
print(f"  - 对 '{tokens[0]}' 的关注分数: {first_layer_first_head[1, 0].item():.4f}")
print(f"  - 对 '{tokens[1]}' 的关注分数: {first_layer_first_head[1, 1].item():.4f}")
print(f"  - 对 '{tokens[2]}' 的关注分数: {first_layer_first_head[1, 2].item():.4f}")
print("\n分数越大，表示关注程度越高！")

输入句子: 'I love Python'
分词结果: ['I', 'Ġlove', 'ĠPython']

第1层、第1个注意力头的注意力分数（未归一化）
含义：每一行表示该 token 对所有 token 的关注程度

                     I       Ġlove     ĠPython
         I      1.0000      0.0000      0.0000
     Ġlove      0.8994      0.1006      0.0000
   ĠPython      0.5894      0.1430      0.2675

举例说明：
当模型处理 'Ġlove' 这个词时：
  - 对 'I' 的关注分数: 0.8994
  - 对 'Ġlove' 的关注分数: 0.1006
  - 对 'ĠPython' 的关注分数: 0.0000

分数越大，表示关注程度越高！


## 📚 总结

### `config.output_attentions = True` 的作用：

1. **默认情况（False）**：
   - 模型只返回最终预测结果（logits）
   - 就像老师只告诉你答案，不告诉你解题过程
   
2. **设置为 True**：
   - 模型会额外返回所有层、所有注意力头的注意力分数
   - 就像老师不仅给你答案，还详细讲解每一步是怎么想的
   
3. **什么时候需要设置为 True？**
   - ✅ 想研究模型是如何"理解"句子的
   - ✅ 想可视化模型的注意力模式
   - ✅ 想调试模型或分析模型行为
   - ❌ 只想要预测结果（设置为 True 会增加内存消耗和计算时间）

### 类比理解：
想象你在用翻译软件翻译 "I love Python"：
- **output_attentions = False**：只给你翻译结果 "我爱 Python"
- **output_attentions = True**：给你翻译结果 + 解释（比如翻译 "love" 时，软件考虑了前面的 "I" 和后面的 "Python"）

---

### 实际应用场景：

在您的 `GroupedQueryAttention.ipynb` 文件中：
```python
# 需要设置 config.output_attentions = True
config = AutoConfig.from_pretrained("gpt2")
config.output_attentions = True
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2", config=config)

# 然后才能访问 attentions
outputs = gpt2(inputs.input_ids)
first_layer_attentions = outputs.attentions[0][0]  # ✅ 不会报错
```

如果不设置，`outputs.attentions` 会是一个包含 None 值的元组，导致 `TypeError: 'NoneType' object is not subscriptable` 错误。